<center><img src="images/header.png"></center>

<h1><center>Лекция №4: Введение в машинное обучение</center></h1>
<hr>
<h2><center>Методы обучения без учителя: Кластеризация (практика)</center></h2>
<h3><center>Шестаков Андрей</center></h3>

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-talk')
plt.rcParams['figure.figsize'] = (12,8)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

In [ ]:
# Uncomment if you are using colab
# !mkdir ./data
# !wget https://github.com/shestakoff/sphere-ml-intro/blob/master/lecture04-unsupervised/data/diet.csv -O ./data/diet.csv
# !wget https://github.com/shestakoff/sphere-ml-intro/blob/master/lecture04-unsupervised/data/snsdata.csv -O ./data/snsdata.csv

# Применение K-means

Загрузите [данные](https://github.com/brenden17/sklearnlab/blob/master/facebook/snsdata.csv) в которых содержится описание интересов профилей учеников старшей школы США.

In [ ]:
df_sns = pd.read_csv('data/snsdata.csv', sep=',')
df_sns.head()

Данные устроены так: 
* Год выпуска
* Пол
* Возраст
* Количество друзей
* 36 ключевых слов, которые встречаются в профилe facebook (интересы, сообщества, встречи)

#### Задание

* Удалите все признаки кроме 36 ключевых слов.
* Нормализуйте данные - из каждого столбца вычтите его среднее значение и поделите на стандартное отклонение.
* Используйте метод k-means чтобы выделить 9 кластеров
* Попробуйте проинтерпретировать каждый кластер проанализировав полученные центройды (Некоторые кластеры могут быть очень большие и очень маленькие - плохо интерпретируются)

In [ ]:
## Your Code Here

# Рационы питания в странах

Загрузите набор данных о пищевом рационе в разных странах мира `diet.csv`

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score

In [ ]:
df = pd.read_csv('data/diet.csv', sep=';').iloc[:, :-1]

In [ ]:
df.head()

In [ ]:
scaler = ...


1. Отнормируйте данные с помощью `RobustScaler` или `StandardScaler`
2. Используйте метод K-средних. Выберите число кластеров с помощью критерия силуэта
3. Найдите выборосы и проинтерпретируйте кластеры

In [ ]:
## Your Code Here

## Оценка `epsilon` для DBSCAN

То что нам не нужно определять количество выходных кластеров в DBSCAN - это конечно хорошо, но как определить `min_pts` и `epsilon`?) Есть одна методика

Будем считать, что MinPts нам дан свыше (например MinPts = 2). Воспользуемся следующим способом оценки:

* Нормализуем признаки, например с помощью `RobustScaler` или `StandartScaler`
* Расчитаем расстояние до k=MinPts ближайшего соседа каждой точки (класс `NearestNeighbors` и метод `kneighbors`)
* Отсортируем полученный массив и выведите его на график
* По графику будет примерно понятно, сколько точек уйдет в шум, а сколько попадет в полноценный кластер

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
## Your Code Here

# Задача про кластеризацию текстов (ДЗ)

Рассмотрим коллекцию новостных сообщений за первую половину 2017 года. Про каждое новостное сообщение известны:
* его заголовок и текст
* дата его публикации
* событие, о котором это новостное сообщение написано 
* его рубрика 

In [ ]:
df = pd.read_csv('./data/news.csv', encoding='utf8')
df.head()

In [ ]:
df.loc[:, 'class'].value_counts()

Попробуем кластеризовать документы (каким-либо методом) и сравним полученное разбиение с данными рубликами с помощью ARI

### Стандартная предобработка текстов

In [ ]:
import re

# Оставляем только кириллические символы
regex = re.compile(u"[А-Яа-я]+")

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))


df.text = df.text.str.lower()
df.loc[:, 'text'] = df.text.apply(words_only)

In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords

# Удаляем стоп-слова
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', u'также',  'т', 'д', '-', '-']

def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return u" ".join([token for token in text.split() if not token in mystopwords])
    except:
        return u""
    
df.text = df.text.apply(remove_stopwords)   

In [ ]:
%%time 
from pymystem3 import Mystem


# нормализуем текст
m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

df.text = df.text.apply(lemmatize)

In [ ]:
mystoplemmas = [u'который', u'прошлый', u'сей', u'свой', u'наш', u'мочь']

# Еще кое-что удаляем
def  remove_stoplemmas(text, mystoplemmas = mystoplemmas):
    try:
        return " ".join([token for token in text.split() if not token in mystoplemmas])
    except:
        return ""

df.text = df.text.apply(remove_stoplemmas)  

In [ ]:
df.text[0]

## Вычисление сходства

С помощью `TfidfVectorizer` и `pairwise_distances` расчитайте косинусное расстояние между всеми парами документов к корпусе

Запишите результат в переменную `S`

In [ ]:
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

In [ ]:
vect = TfidfVectorizer()
texts = df.text.values

In [ ]:
# Your Code Here

In [ ]:
S = ...

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(data=S, cmap = 'Spectral').set(xticklabels=[],yticklabels=[])


Вы должны пронаблюдать, что между некоторыми текстами есть довольно выскокое сходство по содержанию слов. 

Попробуем их кластеризовать

## DBSCAN или KMeans
* Воспользуйтесь методикой оценки параметров для алгоритма DBSCAN. Используйте косинусную меру близости
* Выделите кластеры. Для каждого кластера (кроме -1, если он будет) выведите несколько текстов и умозрительно определите его тематику
* Оцените сходство с изначальными рубриками визуально (с помощью матрицы перемешивания) и с помощью ARI

In [ ]:
# Your Code Here

In [ ]:
labels = ...

In [ ]:
adjusted_rand_score(true_label, labels)

In [ ]:
pd.crosstab(df.loc[:, 'class'], labels)